In [1]:
import json
import random
import numpy as np

In [2]:
with open('user2video.json', 'r') as f:
    uvdict = json.load(f)

In [55]:
user2index = dict()
for user in uvdict.keys():
    user2index[user] = len(user2index)
len_user = len(uvdict)
vid = set()
for vlist in uvdict.values():
    vid.update(set(vlist))
len_vid = len(vid)
len_user, len_vid

(492, 1344)

In [4]:
R_total = np.zeros([len_user, len_vid])

video2index = dict()
for u, v_list in uvdict.items():
    for v in v_list:
        if video2index.get(v) == None:
            video2index[v] = len(video2index)
        R_total[user2index[u], video2index[v]] = 1

In [5]:
R_total

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [39]:
def update(R, U, V, lambda_coef, K, users, items):
    for i in range(users): 
        V_j = V[:, R[i, :] > 0]
        U[:, i] = np.dot(np.linalg.inv(np.dot(V_j, V_j.T) + lambda_coef * np.identity(K)), np.dot(R[i, R[i, :] > 0], V_j.T))
    for j in range(items): 
        U_i = U[:, R[:, j] > 0] 
        V[:, j] = np.dot(np.linalg.inv(np.dot(U_i, U_i.T) + lambda_coef * np.identity(K)), np.dot(R[R[:, j] > 0, j], U_i.T))
    return U,V

def train(R, max_round):
    rounds = 0
    global U,V,K,users,items,lambda_coef
    while rounds<=max_round:
        rounds = rounds + 1    
        U,V=update(R,U,V,lambda_coef,K,users,items)
    return U, V

In [40]:
R_train = R_total.copy()
R_test = list()
for i in range(R_total.shape[0]):
    for j in range(R_total.shape[1]):
        if R_train[i, j] == 1:
            if random.random() < 0.1:
                R_train[i, j] = 0
                R_test.append((i,j))

In [51]:
K = 5
users, items = R_train.shape
lambda_coef = 0.05
U = np.ones([K, users]) / users
V = np.ones([K, items]) / items
U, V=train(R_train, 50)
M = np.round(np.dot(U.T,V))

In [52]:
np.sum(np.abs(M-R_total)) / R_total.size

0.06782326751838946

In [53]:
sm = 0
for (i,j) in R_test:
    sm += abs(M[i,j]-R_total[i,j])
sm / len(R_test)

0.18930957683741648